In [ ]:
import pandas as pd
import pdfplumber
import os

# Define paths
pdf_folder = "/Users/PDFS/"  # Folder containing the PDF files
csv_output = "/Users/combined_data_sorted.csv" #Path you save the data file

# List of all PDF files in the folder
pdf_files = [f for f in os.listdir(pdf_folder) if f.endswith(".pdf")]

# Initialize a list to store data for all months
all_months_data = []

# Month order dictionary for replacing month numbers with names
month_order = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 
    'May': 5, 'June': 6, 'July': 7, 'August': 8, 
    'September': 9, 'October': 10, 'November': 11, 'December': 12
}

# Process each PDF file
for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder, pdf_file)

    # Extract data from the current PDF
    with pdfplumber.open(pdf_path) as pdf:
        all_data = [row for page in pdf.pages for table in page.extract_tables() for row in table]

    # Convert the extracted data into a DataFrame
    df = pd.DataFrame(all_data).drop_duplicates()       #Delete the extra headers
    df.set_index([df.columns[0], df.columns[1]], inplace=True)  #Set the first two columns as index

    # Define multi-level columns
    columns = [
        ('Human Trafficking', 'Sub'), ('Human Trafficking', '%'), ('Human Trafficking', 'UnSub'), ('Human Trafficking', '%'),
        ('Sexual Abuse', 'Sub'), ('Sexual Abuse', '%'), ('Sexual Abuse', 'UnSub'), ('Sexual Abuse', '%'),
        ('Physical Abuse', 'Sub'), ('Physical Abuse', '%'), ('Physical Abuse', 'UnSub'), ('Physical Abuse', '%'),
        ('Neglect', 'Sub'), ('Neglect', '%'), ('Neglect', 'UnSub'), ('Neglect', '%'),
        ('Totals', 'AllSub'), ('Totals', '%'), ('Totals', 'AllUnSub'), ('Totals', '%')
    ]
    df.columns = pd.MultiIndex.from_tuples(columns)

    # Extract data for locations and regions
    locations = ['Carroll', 'Clinton', 'Tippecanoe', 'Howard', 'Blackford', 'Delaware', 'Grant',
                 'Boone', 'Montgomery', 'Shelby', 'Hamilton', 'Hancock', 'Madison', 'Tipton',
                 'Henry', 'Johnson']
    regions = ['Statewide', 'Region 10']

    value_all = pd.concat([
        df.loc[pd.IndexSlice[:, locations], ('Sexual Abuse', 'Sub')],
        df.loc[(regions, 'Total'), ('Sexual Abuse', 'Sub')]
    ])

    # Replace 'Total' in the second-level index with specific names
    value_all.index = value_all.index.map(
        lambda x: (
            x[0],
            x[0] if x[1] == 'Total' and x[0] == 'Statewide' else
            'Marion Total' if x[1] == 'Total' and x[0] == 'Region 10' else
            x[1]
        )
    )
    value_all.index = value_all.index.droplevel(0)      #Delete the first level index (the first column)

    # Rearrange rows: move 'Statewide' and 'Marion Total' to specific positions
    rows_to_move = value_all.loc[["Statewide", "Marion Total"]]
    value_all = value_all.drop(index=["Statewide", "Marion Total"])

    position_1, position_2 = value_all.index.get_loc("Carroll"), value_all.index.get_loc("Shelby") + 1
    value_all = pd.concat([
        value_all.iloc[:position_1], rows_to_move.iloc[[0]],
        value_all.iloc[position_1:position_2], rows_to_move.iloc[[1]],
        value_all.iloc[position_2:]
    ])

    # Transpose and reset index
    value_all = value_all.T.reset_index(drop=True)

    # Extract year and month from the file name
    # Assume file name format is like: "Assessment_Decisions_Summary_by_Child-YYYY-MM.pdf" Please rename your pdf files like this.
    year, month = pdf_file.split('-')[-2], pdf_file.split('-')[-1].replace('.pdf', '')

    # Add Year and Month columns
    value_all.insert(0, "Year", year)
    value_all.insert(1, "Month", month)

    # Append data to the combined list
    all_months_data.append(value_all)

# Combine all data into a single DataFrame
combined_df = pd.concat(all_months_data, ignore_index=True)

# Convert the 'Month' column to an integer type for sorting
combined_df['Month'] = combined_df['Month'].astype(int)

# Sort by Year and Month_Num to get the correct chronological order
combined_df_sorted = combined_df.sort_values(by=['Year', 'Month'])

# Replace the month number with the name of the month
combined_df_sorted['Month'] = combined_df_sorted['Month'].map({v: k for k, v in month_order.items()})

# Print the data frame
print(combined_df_sorted)
# Save the sorted data to a CSV file
combined_df_sorted.to_csv(csv_output, index=False)


/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:51: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[pd.IndexSlice[:, locations], ('Sexual Abuse', 'Sub')],
/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:52: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[(regions, 'Total'), ('Sexual Abuse', 'Sub')]
/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:51: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[pd.IndexSlice[:, locations], ('Sexual Abuse', 'Sub')],
/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:52: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[(regions, 'Total'), ('Sexual Abuse', 'Sub')]
/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:51: PerformanceWarning: indexing past lexsort depth may impact pe

1  Year      Month Statewide Carroll Clinton Tippecanoe Howard Blackford  \
4  2024        May       167       0       1          3      2         1   
5  2024       June       186       1       1          0      1         1   
1  2024       July       146       1       2          0      1         1   
3  2024     August       182       1       0          4      4         0   
2  2024  September       207       1       0          3      6         0   
0  2024    October       184       0       0          1      1         0   

1 Delaware Grant Boone Montgomery Shelby Marion Total Hamilton Hancock  \
4        4     4     0          0      1           19        3       0   
5        3     5     2          2      0           22        1       1   
1        2     0     1          2      0           19        1       0   
3        2     4     1          1      1           22        1       4   
2        4     1     2          2      0           24        1       6   
0        3     5     0 

/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:51: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[pd.IndexSlice[:, locations], ('Sexual Abuse', 'Sub')],
/var/folders/d0/rr6p1rqx4pv_99nns363nzyr0000gn/T/ipykernel_68281/1223681203.py:52: PerformanceWarning: indexing past lexsort depth may impact performance.
  df.loc[(regions, 'Total'), ('Sexual Abuse', 'Sub')]
